In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from abc import ABCMeta
import argparse
import datetime 
import collections
from collections import defaultdict
import json
import math
import os
import random
import pickle
import sys
import io
from tqdm import tqdm, trange
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.nn import CrossEntropyLoss
from pathlib import Path
from collections import Counter

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering
from pytorch_pretrained_bert.modeling import BertForQuestionAnsweringQC4QA

from pytorch_pretrained_bert.optimization import BertAdam
from utils.ConfigLogger import config_logger
from utils.evaluate import f1_score, exact_match_score, metric_max_over_ground_truths
from utils.BERTRandomSampler import BERTRandomSampler

PYTORCH_PRETRAINED_BERT_CACHE = Path(os.getenv('PYTORCH_PRETRAINED_BERT_CACHE',
                                               Path.home() / '.pytorch_pretrained_bert'))

from da_data_utils import * # Question Type용 Data_utils 파일(원본은 data_utils.py)
from qada_utils import *
############################
import importlib, types, argparse
from utils.ConfigLogger import config_logger

In [2]:
train_squad_path = '../../../data/squad/train-v1.1_classified_qtype_prob.jsonl'
dev_squad_path = '../../../data/squad/dev-v1.1.json'
train_cnn_path = '../../../data/cnn/cnn_train_classified_qtype_prob.jsonl'
dev_cnn_path = '../../../data/cnn/cnn_dev.json'


def open_jsonl(path):
    data = []
    with io.open(path, 'r', encoding='utf-8') as f:
        for example in f:
            data.append(json.loads(example))
    data = data[:10] # debug size
    return data

def open_json(path):
    with io.open(path, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
    input_data = input_data[:10]  # debug size
    return input_data

train_squad_data = open_jsonl(train_squad_path)
dev_squad_data = open_json(dev_squad_path)
train_cnn_data = open_jsonl(train_cnn_path)
dev_cnn_data = open_json(dev_cnn_path)

In [3]:
train_squad_data[0]['paragraphs'][0]['qas'][0]

{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}],
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'id': '5733be284776f41900661182',
 'q_type': 3,
 'q_type_prob': [0.0, 0.0, 0.016, 0.983, 0.001, 0.0]}

In [4]:
train_cnn_data[0]['paragraphs'][0]['qas'][0]

{'answers': [{'answer_start': 2070, 'text': 'Globe'}],
 'question': 'on the Democratic side , the Register backs Clinton , while the @placeholder picks Obama',
 'id': 'training/6dc32db9379e43971ad93007e76c5347e213f21a',
 'q_type': 3,
 'q_type_prob': [0.001, 0.001, 0.171, 0.76, 0.064, 0.002]}

In [5]:
# parameters
args = argparse.Namespace(
    debug = True, # sample_limit 사이즈로 디버깅
    bert_model='bert-base-uncased',
    do_adaptation=True,
    do_predict=False,
    do_lower_case=True,
    source_train_file="../../../data/squad/train-v1.1_classified_qtype_prob.jsonl",
    target_train_file="../../../data/cnn/cnn_train_classified_qtype_prob.jsonl",
    target_predict_file="../../../data/cnn/cnn_dev.json",
    input_dir="model/squad",
    input_model_file="best_model_0916.bin",
    output_dir="model/squad2target",
    output_model_file="adaptation_1031.bin",
    logger_path="model/squad2target",
    max_seq_length=512,
    seed=42,
    gradient_accumulation_steps=1,
    train_batch_size=12,
    predict_batch_size=12,
    num_workers=4,
    evaluation_interval=2000,
    loss_logging_interval=500,
    train_learning_rate=3e-5,
    num_train_epochs=2,
    warmup_proportion=0.1,
    n_best_size=20,
    max_answer_length=30,
    verbose_logging=False,
    use_simple_feature=False,
    generate_prob_th=0.6,
    keep_previous_generated=False,
    use_BN=True,
    output_prediction=True,
    source_sampling_ratio=3,
    doc_stride=128,
    max_query_length=64,
    adapt_learning_rate=1e-5,
    lambda_c=0.1,
    sample_limit= 100  ### 디버깅용 데이터셋 크기
)

In [ ]:
def prediction_stage(args, device, tokenizer, logger, debug=False):
    # Load a trained model that you have fine-tuned
    output_model_file = os.path.join(args.output_dir, args.output_model_file)
    model_state_dict = torch.load(output_model_file)
    model = BertForQuestionAnswering.from_pretrained(args.bert_model, state_dict=model_state_dict, args=args)
    model.to(device)
    # Read prediction samples
    read_limit = None
    if debug:
        read_limit = 100 # 샘플 100개만 사용
    logger.info("***** Reading Prediction Samples *****")
    eval_features, eval_examples = read_features_and_examples(args, args.predict_file, tokenizer, logger,
            use_simple_feature=False, read_examples=True, limit=read_limit)
    acc, f1 = evaluation_stage(args, eval_examples, eval_features, device, model, logger)
    logger.info('***** Prediction Performance *****')
    logger.info('EM is %.5f, F1 is %.5f', acc, f1)


def evaluate_acc_and_f1(predictions, raw_data, logger, threshold=-1, all_probs=None):
    f1 = exact_match = total = 0
    eval_threshold = True
    if threshold is None or all_probs is None:
        eval_threshold = False
    for sample in raw_data:
        if (sample.qas_id not in predictions) or (eval_threshold and sample.qas_id not in all_probs):
            message = 'Unanswered question ' + sample.qas_id + ' will receive score 0.'
            logger.warn(message)
            continue
        if not eval_threshold or (eval_threshold and all_probs[sample.qas_id] >= threshold):
            ground_truths = sample.orig_answers
            prediction = predictions[sample.qas_id]
            exact_match += metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            f1 += metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)
            total += 1

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return exact_match, f1


def keep_high_prob_samples(all_probs, all_features, prob_threshold, removed_feature_index, all_indices,
        keep_generated=False):
    '''
    셀프 트레이닝용: 높은 확률의 예측을 pseudo-label로 변환
    '''
    new_train_features = []
    for i, feature in enumerate(all_features):
        if keep_generated:
            if feature.example_index not in removed_feature_index and all_probs[feature.example_index] > prob_threshold:
                feature.start_position, feature.end_position = all_indices[i][0] = all_indices[i][1]
                new_train_features.append(feature)
                removed_feature_index.add(feature.example_index)
        else:
            if all_probs[feature.example_index] > prob_threshold:
                feature.start_position, feature.end_position = all_indices[i][0], all_indices[i][1]
                new_train_features.append(feature)
    return new_train_features, removed_feature_index


def compare_performance(args, best_acc, best_f1, acc, f1, model, logger):
    if not (best_f1 is None or best_acc is None):
        if best_acc < acc:
            logger.info('Current model BEATS previous best model, previous best is EM = %.5F, F1 = %.5f',
                best_acc, best_f1)
            best_acc, best_f1 = acc, f1
            logger.info('Current best model has been saved!')
            model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
            torch.save(model_to_save.state_dict(), os.path.join(args.output_dir, args.output_model_file))
        else:
            logger.info('Current model CANNOT beat previous best model, previous best is EM = %.5F, F1 = %.5f',
                best_acc, best_f1)
    else:
        best_acc, best_f1 = acc, f1
    return best_acc, best_f1


def evaluation_stage(args, eval_examples, eval_features, device, model, logger, generate_prob_th=0.6,
        removed_feature_index=None, global_step=None, best_acc=None, best_f1=None, generate_label=False):
    if not global_step:
        logger.info("***** Running Evaluation Stage *****")
    else:
        logger.info("***** Running Predictions *****")
    logger.info("  Num orig examples = %d", len(eval_examples))
    logger.info("  Num split examples = %d", len(eval_features))
    logger.info("  Batch size = %d", args.predict_batch_size)

    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.predict_batch_size)

    model.eval()
    all_results = []
    logger.info("Start evaluating")
    for input_ids, input_mask, segment_ids, example_indices in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        with torch.no_grad():
             batch_start_logits, batch_end_logits, _ = model(input_ids, segment_ids, input_mask)
        for i, example_index in enumerate(example_indices):
            start_logits = batch_start_logits[i].detach().cpu().tolist()
            end_logits = batch_end_logits[i].detach().cpu().tolist()
            eval_feature = eval_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            all_results.append(RawResult(unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits))

    if global_step:
        prediction_file_name = 'predictions_' + str(global_step) + f'_{datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S")}.json'
        nbest_file_name = 'nbest_predictions_' + str(global_step) + f'_{datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S")}.json'
        output_prediction_file = os.path.join(args.output_dir, prediction_file_name)
        output_nbest_file = os.path.join(args.output_dir, nbest_file_name)
    else:
        output_prediction_file = os.path.join(args.output_dir, f'predictions_{datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S")}.json')
        output_nbest_file = os.path.join(args.output_dir, f'nbest_predictions_{datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S")}.json')
    all_predictions, all_probs, all_indices = write_predictions(args, eval_examples, eval_features, all_results,
        args.n_best_size, args.max_answer_length,
        args.do_lower_case, output_prediction_file,
        output_nbest_file, args.verbose_logging, logger, args.output_prediction)
    if generate_label:
        return keep_high_prob_samples(all_probs, eval_features, generate_prob_th, removed_feature_index, all_indices,
                keep_generated=args.keep_previous_generated)
    else:
        acc, f1 = evaluate_acc_and_f1(all_predictions, eval_examples, logger)
        logger.info('Current EM is %.5f, F1 is %.5f', acc, f1)
        if not (best_f1 is None or best_acc is None):
            best_acc, best_f1 = compare_performance(args, best_acc, best_f1, acc, f1, model, logger)
            return best_acc, best_f1
        else:
            return acc, f1


def generate_self_training_samples(args, train_examples, train_features, device, model, removed_feature_index,
        new_generated_train_features, generate_prob_th, logger):
    '''
    타겟 도메인 데이터에서 pseudo-label 생성
    '''
    logger.info('***** Generating training data for this epoch *****')
    if args.keep_previous_generated:
        train_features_removed_previous = []
        for index in range(len(train_features)):
            if index not in removed_feature_index:
                train_features_removed_previous.append(train_features[index])
    else:
        train_features_removed_previous = train_features
    cur_train_features, removed_feature_index = \
        evaluation_stage(args, train_examples, train_features_removed_previous, device, model, logger,
            removed_feature_index=removed_feature_index, generate_label=True, generate_prob_th=generate_prob_th)
    if len(cur_train_features) == 0:
        logger.info("  No new training samples were generated, training procedure ends")
        return None, None
    if args.keep_previous_generated:
        new_generated_train_features.extend(cur_train_features)
    else:
        new_generated_train_features = cur_train_features
    return new_generated_train_features, removed_feature_index


def get_bert_model_parameters(model):
    '''
    역할:BERT optimizer 파라미터 그룹 생성 (weight decay 적용/미적용 분리)
    반환: optimizer_grouped_parameters
    '''
    # Prepare optimizer
    param_optimizer = list(model.named_parameters())

    # hack to remove pooler, which is not used
    # thus it produce None grad that break apex
    param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    return optimizer_grouped_parameters


def comb_adversarial_training_stage(args, target_train_features, target_train_examples, source_train_features,
            eval_features, eval_examples, removed_feature_index, new_generated_train_features, model, epoch,
            device, best_acc, best_f1, logger):
    '''
    1. Self-training: 타겟 데이터에서 pseudo-label 생성
    2. Question Type Probability 기반 Source-Target 매칭:
        타겟 배치의 각 샘플에서 q_type_prob 상위 3개를 추출하여
        각 타입에 해당하는 소스 샘플을 샘플링 (타겟 1개 → 소스 3개)
    3. Combined Training: 타겟(pseudo) + 소스(labeled) 데이터를 concat하여 학습
    '''

    def sample_source_batch_top3(source_data, source_q_type_dict, sample_pointer, q_type_probs):
        """
        q_type_probs 기반 상위 3개 타입에서 소스 샘플링
        Args:
            q_type_probs: [batch_size, 6] 텐서
        Returns:
            타겟 배치의 최대 3배 크기 소스 배치
        """
        output_idx = []

        for q_type_prob in q_type_probs:  # 각 타겟 샘플
            # 상위 3개 q_type 인덱스 추출
            top3_indices = torch.topk(q_type_prob, k=3).indices.tolist()

            for q_type in top3_indices:
                # 해당 타입에 소스 샘플이 없으면 스킵
                if len(source_q_type_dict[q_type]) == 0:
                    continue

                next_q_idx = sample_pointer[q_type] % len(source_q_type_dict[q_type])
                output_idx.append(source_q_type_dict[q_type][next_q_idx])
                sample_pointer[q_type] += 1

        input_ids_source, input_masks_source, segment_ids_source, start_positions_source, end_positions_source, \
            q_types_source = [], [], [], [], [], []
        for idx in output_idx:
            input_ids_source.append(source_data[idx][0].unsqueeze(0))
            input_masks_source.append(source_data[idx][1].unsqueeze(0))
            segment_ids_source.append(source_data[idx][2].unsqueeze(0))
            start_positions_source.append(source_data[idx][3].unsqueeze(0))
            end_positions_source.append(source_data[idx][4].unsqueeze(0))
            q_types_source.append(source_data[idx][5].unsqueeze(0))

        return torch.vstack(input_ids_source), torch.vstack(input_masks_source), torch.vstack(segment_ids_source), \
            torch.cat(start_positions_source, -1), torch.cat(end_positions_source, -1), torch.cat(q_types_source, -1)

    # Generate self-training samples
    # 1. Pseudo-label 생성
    new_generated_train_features, removed_feature_index = generate_self_training_samples(args, target_train_examples,
        target_train_features, device, model, removed_feature_index, new_generated_train_features, args.generate_prob_th,
        logger)
    if new_generated_train_features is None:
        sys.exit()
    
    logger.info('\n')
    logger.info('====================  Start Adversarial Training Stage  ====================')
    
    # q_type_prob 추출 (데이터에서 가져오기)
    all_q_type_probs = []
    for f in new_generated_train_features:
        # InputFeatures에 q_type_prob가 있는지 확인
        if hasattr(f, 'q_type_prob') and f.q_type_prob is not None:
            all_q_type_probs.append(f.q_type_prob)
        else:
            # q_type_prob가 없으면 one-hot 인코딩 사용
            prob = [0.0] * 6
            prob[f.q_type] = 1.0
            all_q_type_probs.append(prob)
    
    all_input_ids = torch.tensor([f.input_ids for f in new_generated_train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in new_generated_train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in new_generated_train_features], dtype=torch.long)
    all_start_positions = torch.tensor([f.start_position for f in new_generated_train_features], dtype=torch.long)
    all_end_positions = torch.tensor([f.end_position for f in new_generated_train_features], dtype=torch.long)
    all_q_types = torch.tensor([f.q_type for f in new_generated_train_features], dtype=torch.long)
    all_q_type_probs = torch.tensor(all_q_type_probs, dtype=torch.float)
    
    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
        all_start_positions, all_end_positions, all_q_types, all_q_type_probs)
    
    source_input_ids = torch.tensor([f.input_ids for f in source_train_features], dtype=torch.long)
    source_input_mask = torch.tensor([f.input_mask for f in source_train_features], dtype=torch.long)
    source_segment_ids = torch.tensor([f.segment_ids for f in source_train_features], dtype=torch.long)
    source_start_positions = torch.tensor([f.start_position for f in source_train_features], dtype=torch.long)
    source_end_positions = torch.tensor([f.end_position for f in source_train_features], dtype=torch.long)
    source_q_types = []
    source_q_type_dict = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: []
    }
    for idx, f in enumerate(source_train_features):
        source_q_types.append(f.q_type)
        source_q_type_dict[f.q_type].append(idx)
    source_q_types = torch.tensor(source_q_types, dtype=torch.long)
    for key in source_q_type_dict.keys():
        random.shuffle(source_q_type_dict[key])
    sample_pointer = [0] * 6
    source_data = TensorDataset(source_input_ids, source_input_mask, source_segment_ids, source_start_positions, 
        source_end_positions, source_q_types)

    train_sampler = BERTRandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)
    
    data_len = len(new_generated_train_features)
    logger.info("  Num split examples = %d", data_len)
    logger.info("  Batch size = %d", args.train_batch_size)
    num_train_steps = math.ceil(data_len / args.train_batch_size / args.gradient_accumulation_steps)
    if num_train_steps == 0 and data_len > 0:
        num_train_steps = 1
    t_total = num_train_steps
    logger.info("  Num steps = %d", num_train_steps)

    loss_sum = 0
    optimizer_grouped_parameters = get_bert_model_parameters(model)
    optimizer = BertAdam(optimizer_grouped_parameters,
            lr=args.adapt_learning_rate,
            warmup=args.warmup_proportion,
            t_total=t_total)
    global_step = 0

    # 타겟 배치 순회(pseudo-labeling->소스 매칭->결합 학습)
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        model.train()
        # 소스 배치 샘플링 (question type probability 기반 상위 3개)
        batch_source = sample_source_batch_top3(source_data, source_q_type_dict, sample_pointer, batch[-1])
        batch = tuple(t.to(device) for t in batch[:-1])  # q_type_probs는 제외
        batch_source = tuple(t.to(device) for t in batch_source)
        input_ids, input_masks, segment_ids, start_positions, end_positions, q_types = batch
        input_ids_source, input_masks_source, segment_ids_source, start_positions_source, end_positions_source, q_types_source = batch_source
        # 타겟 + 소스 concat
        input_ids = torch.cat((input_ids, input_ids_source), 0)
        input_masks = torch.cat((input_masks, input_masks_source), 0)
        segment_ids = torch.cat((segment_ids, segment_ids_source), 0)
        start_positions = torch.cat((start_positions, start_positions_source), 0)
        end_positions = torch.cat((end_positions, end_positions_source), 0)
        q_types = torch.cat((q_types, q_types_source), 0)
        # QC4QA loss 계산 및 역전파
        loss = model.forward_ours(input_ids, segment_ids, input_masks, start_positions,
                end_positions, q_types, lambda_c=args.lambda_c)
        if args.gradient_accumulation_steps > 1:
            loss = loss / args.gradient_accumulation_steps
        
        loss.backward()
        
        if (step + 1) % args.gradient_accumulation_steps == 0:
            # modify learning rate with special warm up BERT uses
            lr_this_step = args.adapt_learning_rate * warmup_linear(global_step / t_total, args.warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        
    final_acc, final_f1 = None, None
    if epoch == args.num_train_epochs - 1:
        final_acc, final_f1 = evaluation_stage(args, eval_examples, eval_features, device, model,
            global_step=global_step, best_acc=None, best_f1=None, logger=logger)
        best_acc, best_f1 = compare_performance(args, best_acc, best_f1, final_acc, final_f1, model, logger)
    else:
        best_acc, best_f1 = evaluation_stage(args, eval_examples, eval_features, device, model,
            global_step=global_step, best_acc=best_acc, best_f1=best_f1, logger=logger)
    return best_acc, best_f1, final_acc, final_f1


def prepare_model(args, device):
    # Source 도메인에서 fine-tuned 모델 로드(run_source.py는 사전학습 모델 로드)
    input_model_file = os.path.join(args.input_dir, args.input_model_file)
    model_state_dict = torch.load(input_model_file)
    model = BertForQuestionAnsweringQC4QA.from_pretrained(args.bert_model, state_dict=model_state_dict, args=args)
    model.to(device)
    return model


def adaptation_stage(args, tokenizer, device, logger, debug=True):
    '''
    역할: 도메인 적응 메인 루프
    '''
    ### 데이터 수 조절 ### 
    sample_limit = 100
    
    model = prepare_model(args, device)
    best_acc, best_f1 = 0, 0
    
    read_limit = None
    if debug:
        read_limit = 50

    ## Read target training examples
    logger.info("***** Reading Target Unlabeled Training Samples *****")
    train_features, train_examples = read_features_and_examples(args, args.target_train_file, tokenizer, logger,
        use_simple_feature=False, read_examples=True, limit=read_limit)

    ## Read source training examples
    logger.info("***** Reading Source Training Samples *****")
    source_train_features, _ = read_features_and_examples(args, args.source_train_file, tokenizer, logger,
        use_simple_feature=False, read_examples=True, limit=read_limit)

    # Read evaluation samples
    logger.info("***** Reading Evaluation Samples *****")
    eval_features, eval_examples = read_features_and_examples(args, args.target_predict_file, tokenizer, logger,
        use_simple_feature=False, read_examples=True, limit=read_limit)

    removed_feature_index = set()
    new_generated_train_features = []
    final_acc, final_f1 = 0.0, 0.0
    for epoch in trange(int(args.num_train_epochs), desc="Epoch"):
        logger.info('\n')
        logger.info(' ###########  Start Training Epoch %d  ###########', epoch + 1)
        logger.info('\n')
        best_acc, best_f1, final_acc, final_f1 = comb_adversarial_training_stage(args, train_features, train_examples,
                source_train_features, eval_features, eval_examples, removed_feature_index, new_generated_train_features,
                model, epoch, device, best_acc, best_f1, logger)
        logger.info('\n')
        logger.info(' ###########  End Training Epoch %d  ###########', epoch + 1)
        logger.info('\n')

    # Save the final trained model
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    output_model_file = os.path.join(args.output_dir, args.output_model_file + '.final')
    torch.save(model_to_save.state_dict(), output_model_file)
    logger.info('The final model has been save')
    logger.info('*** The Training Stage is Ended ***')
    logger.info('\n\nBest EM is %.5f. Best F1 is %.5f', best_acc, best_f1)
    logger.info('\n\nFinal EM is %.5f. Best F1 is %.5f', final_acc, final_f1)

In [7]:
# filepath: /home/gayeon39/gayeon/[DA]/preliminary/QC4QA/src/run_qc4qa.ipynb
# 3) 로거/디바이스/토크나이저 준비
logger = config_logger(args.logger_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

print(f"device = {device}")

11/04/2025 14:51:50 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/gayeon39/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


device = cuda


In [8]:
# 4) 입력 모델 로드 및 적응 단계 실행
model = prepare_model(args, device)
# adaptation_stage(args, tokenizer, device, logger, debug=True)

/tmp/ipykernel_2255560/2878386538.py:360: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(input_model_file)
11/04/2025 14:51:51 - INFO - pytorch_

In [9]:
###############################################################################
# 🧪 테스트 코드: 데이터 파이프라인 검증
###############################################################################

print("="*80)
print("🔍 DATA PIPELINE TEST - 데이터 흐름 확인")
print("="*80)

# Step 1: Raw JSON 데이터 확인
print("\n" + "─"*80)
print("📄 STEP 1: Raw JSON Data")
print("─"*80)
sample_qa = train_squad_data[0]['paragraphs'][0]['qas'][0]
print(f"✓ Question: {sample_qa['question'][:60]}...")
print(f"✓ q_type: {sample_qa['q_type']}")
print(f"✓ q_type_prob: {sample_qa['q_type_prob']}")
print(f"✓ Top-3 types: {sorted(enumerate(sample_qa['q_type_prob']), key=lambda x: x[1], reverse=True)[:3]}")

🔍 DATA PIPELINE TEST - 데이터 흐름 확인

────────────────────────────────────────────────────────────────────────────────
📄 STEP 1: Raw JSON Data
────────────────────────────────────────────────────────────────────────────────
✓ Question: To whom did the Virgin Mary allegedly appear in 1858 in Lour...
✓ q_type: 3
✓ q_type_prob: [0.0, 0.0, 0.016, 0.983, 0.001, 0.0]
✓ Top-3 types: [(3, 0.983), (2, 0.016), (4, 0.001)]


In [10]:
# Step 2: SquadExample 생성 확인
print("\n" + "─"*80)
print("📦 STEP 2: SquadExample Creation")
print("─"*80)
test_examples = read_squad_examples(
    input_file='../../../data/squad/train-v1.1_classified_qtype_prob.jsonl',
    is_training=True,
    logger=logger
)
print(f"✓ Total examples: {len(test_examples)}")
print(f"✓ Example[0] type: {type(test_examples[0])}")
print(f"✓ Example[0].q_type: {test_examples[0].q_type}")
print(f"✓ Example[0].q_type_prob: {test_examples[0].q_type_prob}")
print(f"✓ Has q_type_prob attribute: {hasattr(test_examples[0], 'q_type_prob')}")


────────────────────────────────────────────────────────────────────────────────
📦 STEP 2: SquadExample Creation
────────────────────────────────────────────────────────────────────────────────
✓ Total examples: 87599
✓ Example[0] type: <class 'da_data_utils.SquadExample'>
✓ Example[0].q_type: 3
✓ Example[0].q_type_prob: [0.0, 0.0, 0.016, 0.983, 0.001, 0.0]
✓ Has q_type_prob attribute: True


In [ ]:
# Step 3: InputFeatures 변환 확인
print("\n" + "─"*80)
print("🔧 STEP 3: InputFeatures Conversion")
print("─"*80)
test_features = convert_examples_to_features(
    examples=test_examples[:5],  # 5개만 테스트
    tokenizer=tokenizer,
    max_seq_length=args.max_seq_length,
    doc_stride=args.doc_stride,
    max_query_length=args.max_query_length,
    is_training=True,
    logger=logger,
    use_simple_feature=False
)
print(f"✓ Total features: {len(test_features)}")
print(f"✓ Feature[0] type: {type(test_features[0])}")
print(f"✓ Feature[0].q_type: {test_features[0].q_type}")
print(f"✓ Feature[0].q_type_prob: {test_features[0].q_type_prob}")
print(f"✓ Has q_type_prob attribute: {hasattr(test_features[0], 'q_type_prob')}")


In [12]:
# Step 4: TensorDataset 생성 확인
print("\n" + "─"*80)
print("🎯 STEP 4: TensorDataset Creation")
print("─"*80)

# q_type_prob 추출
all_q_type_probs = []
for f in test_features:
    if hasattr(f, 'q_type_prob') and f.q_type_prob is not None:
        all_q_type_probs.append(f.q_type_prob)
    else:
        # Fallback: one-hot encoding
        prob = [0.0] * 6
        prob[f.q_type] = 1.0
        all_q_type_probs.append(prob)

all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_q_types = torch.tensor([f.q_type for f in test_features], dtype=torch.long)
all_q_type_probs = torch.tensor(all_q_type_probs, dtype=torch.float)

print(f"✓ all_input_ids shape: {all_input_ids.shape}")
print(f"✓ all_q_types shape: {all_q_types.shape}")
print(f"✓ all_q_type_probs shape: {all_q_type_probs.shape}")
print(f"✓ Sample q_type: {all_q_types[0].item()}")
print(f"✓ Sample q_type_prob: {all_q_type_probs[0].tolist()}")


────────────────────────────────────────────────────────────────────────────────
🎯 STEP 4: TensorDataset Creation
────────────────────────────────────────────────────────────────────────────────
✓ all_input_ids shape: torch.Size([5, 512])
✓ all_q_types shape: torch.Size([5])
✓ all_q_type_probs shape: torch.Size([5, 6])
✓ Sample q_type: 3
✓ Sample q_type_prob: [0.0, 0.0, 0.01600000075995922, 0.9829999804496765, 0.0010000000474974513, 0.0]


In [13]:
# Step 5: Top-3 추출 테스트
print("\n" + "─"*80)
print("🔝 STEP 5: Top-3 Question Type Extraction")
print("─"*80)

for i in range(min(3, len(all_q_type_probs))):
    q_type_prob = all_q_type_probs[i]
    top3_indices = torch.topk(q_type_prob, k=3).indices.tolist()
    top3_values = torch.topk(q_type_prob, k=3).values.tolist()
    
    print(f"\n  Sample {i}:")
    print(f"    • Original q_type: {all_q_types[i].item()}")
    print(f"    • Full q_type_prob: {q_type_prob.tolist()}")
    print(f"    • Top-3 indices: {top3_indices}")
    print(f"    • Top-3 values: {[f'{v:.3f}' for v in top3_values]}")


────────────────────────────────────────────────────────────────────────────────
🔝 STEP 5: Top-3 Question Type Extraction
────────────────────────────────────────────────────────────────────────────────

  Sample 0:
    • Original q_type: 3
    • Full q_type_prob: [0.0, 0.0, 0.01600000075995922, 0.9829999804496765, 0.0010000000474974513, 0.0]
    • Top-3 indices: [3, 2, 4]
    • Top-3 values: ['0.983', '0.016', '0.001']

  Sample 1:
    • Original q_type: 2
    • Full q_type_prob: [0.0, 0.0010000000474974513, 0.9950000047683716, 0.004000000189989805, 0.0, 0.0]
    • Top-3 indices: [2, 3, 1]
    • Top-3 values: ['0.995', '0.004', '0.001']

  Sample 2:
    • Original q_type: 2
    • Full q_type_prob: [0.0, 0.0010000000474974513, 0.9229999780654907, 0.01899999938905239, 0.05400000140070915, 0.0020000000949949026]
    • Top-3 indices: [2, 4, 3]
    • Top-3 values: ['0.923', '0.054', '0.019']


In [14]:
# Step 6: Source-Target 매칭 시뮬레이션
print("\n" + "─"*80)
print("🔗 STEP 6: Source-Target Matching Simulation")
print("─"*80)

# Mock source data-> value는 source feature의 인덱스 의미
source_q_type_dict = {
    0: [0, 1, 2],      # What 질문 3개
    1: [3, 4],         # When 질문 2개
    2: [5, 6, 7],      # Where 질문 3개
    3: [8, 9, 10, 11], # Who 질문 4개
    4: [12],           # Why 질문 1개
    5: [13, 14]        # How 질문 2개
}

print("Mock Source Distribution:")
for q_type, indices in source_q_type_dict.items():
    print(f"  • Type {q_type}: {len(indices)} samples")

# 타겟 샘플 1개에 대해 매칭 시뮬레이션
target_sample_prob = all_q_type_probs[0]
top3_indices = torch.topk(target_sample_prob, k=3).indices.tolist()

print(f"\nTarget Sample:")
print(f"  • q_type_prob: {target_sample_prob.tolist()}")
print(f"  • Top-3 types: {top3_indices}")

print(f"\nMatched Source Samples (1 target → 3 sources):")
for rank, q_type in enumerate(top3_indices):
    if len(source_q_type_dict[q_type]) > 0:
        source_idx = source_q_type_dict[q_type][0]  # 첫 번째 샘플 선택
        print(f"  • Rank {rank+1}: Type {q_type} → Source sample index {source_idx}")
    else:
        print(f"  • Rank {rank+1}: Type {q_type} → ⚠️ No source samples available")


────────────────────────────────────────────────────────────────────────────────
🔗 STEP 6: Source-Target Matching Simulation
────────────────────────────────────────────────────────────────────────────────
Mock Source Distribution:
  • Type 0: 3 samples
  • Type 1: 2 samples
  • Type 2: 3 samples
  • Type 3: 4 samples
  • Type 4: 1 samples
  • Type 5: 2 samples

Target Sample:
  • q_type_prob: [0.0, 0.0, 0.01600000075995922, 0.9829999804496765, 0.0010000000474974513, 0.0]
  • Top-3 types: [3, 2, 4]

Matched Source Samples (1 target → 3 sources):
  • Rank 1: Type 3 → Source sample index 8
  • Rank 2: Type 2 → Source sample index 5
  • Rank 3: Type 4 → Source sample index 12


In [15]:
# Step 7: 배치 크기 비교
print("\n" + "─"*80)
print("📊 STEP 7: Batch Size Comparison")
print("─"*80)

target_batch_size = 12
source_batch_size_old = target_batch_size  # Before: 1:1 매칭
source_batch_size_new = target_batch_size * 3  # After: 1:3 매칭

print(f"Before (1:1 matching):")
print(f"  • Target batch: {target_batch_size}")
print(f"  • Source batch: {source_batch_size_old}")
print(f"  • Total batch: {target_batch_size + source_batch_size_old}")

print(f"\nAfter (1:3 matching with top-3 q_type_prob):")
print(f"  • Target batch: {target_batch_size}")
print(f"  • Source batch: {source_batch_size_new} (최대)")
print(f"  • Total batch: {target_batch_size + source_batch_size_new} (최대)")
print(f"  • Memory increase: {((target_batch_size + source_batch_size_new) / (target_batch_size + source_batch_size_old) - 1) * 100:.1f}%")

print("\n" + "="*80)
print("✅ DATA PIPELINE TEST COMPLETED")
print("="*80)


────────────────────────────────────────────────────────────────────────────────
📊 STEP 7: Batch Size Comparison
────────────────────────────────────────────────────────────────────────────────
Before (1:1 matching):
  • Target batch: 12
  • Source batch: 12
  • Total batch: 24

After (1:3 matching with top-3 q_type_prob):
  • Target batch: 12
  • Source batch: 36 (최대)
  • Total batch: 48 (최대)
  • Memory increase: 100.0%

✅ DATA PIPELINE TEST COMPLETED


In [17]:
###############################################################################
# 🧪 테스트 코드 2: 실제 학습 배치 시뮬레이션
###############################################################################

print("\n" + "="*80)
print("🎓 TRAINING BATCH SIMULATION - 실제 학습 데이터 확인")
print("="*80)

# 실제 데이터 로드 (소량)
print("\n" + "─"*80)
print("📁 Loading Real Data (Source + Target)")
print("─"*80)
train_squad_path = '../../../data/squad/train-v1.1_classified_qtype_prob.jsonl'
train_cnn_path = '../../../data/cnn/cnn_train_classified_qtype_prob.jsonl'

# Source features 로드
source_features, _ = read_features_and_examples(
    args, 
    args.source_train_file, 
    tokenizer, 
    logger,
    use_simple_feature=False, 
    read_examples=True, 
    limit=20  # 20개만 로드
)

# Target features 로드
target_features, target_examples = read_features_and_examples(
    args,
    args.target_train_file,
    tokenizer,
    logger,
    use_simple_feature=False,
    read_examples=True,
    limit=10  # 10개만 로드
)

print(f"✓ Source features loaded: {len(source_features)}")
print(f"✓ Target features loaded: {len(target_features)}")


🎓 TRAINING BATCH SIMULATION - 실제 학습 데이터 확인

────────────────────────────────────────────────────────────────────────────────
📁 Loading Real Data (Source + Target)
────────────────────────────────────────────────────────────────────────────────
✓ Source features loaded: 20
✓ Target features loaded: 10


In [18]:
# Source q_type 분포 확인
print("\n" + "─"*80)
print("📊 Source Q-Type Distribution")
print("─"*80)

source_q_type_dict = {0: [], 1: [], 2: [], 3: [], 4: [], 5: []}
for idx, f in enumerate(source_features):
    source_q_type_dict[f.q_type].append(idx)

for q_type, indices in source_q_type_dict.items():
    print(f"  • Type {q_type}: {len(indices)} samples")
    
# Target q_type_prob 확인
print("\n" + "─"*80)
print("🎯 Target Q-Type Probability Check")
print("─"*80)

target_q_type_probs = []
for f in target_features:
    if hasattr(f, 'q_type_prob') and f.q_type_prob is not None:
        target_q_type_probs.append(f.q_type_prob)
    else:
        prob = [0.0] * 6
        prob[f.q_type] = 1.0
        target_q_type_probs.append(prob)

print(f"✓ Target samples with q_type_prob: {len(target_q_type_probs)}")
print(f"\nFirst 3 target samples:")
for i in range(min(3, len(target_q_type_probs))):
    print(f"  Sample {i}: {target_q_type_probs[i]}")


────────────────────────────────────────────────────────────────────────────────
📊 Source Q-Type Distribution
────────────────────────────────────────────────────────────────────────────────
  • Type 0: 0 samples
  • Type 1: 0 samples
  • Type 2: 7 samples
  • Type 3: 5 samples
  • Type 4: 1 samples
  • Type 5: 7 samples

────────────────────────────────────────────────────────────────────────────────
🎯 Target Q-Type Probability Check
────────────────────────────────────────────────────────────────────────────────
✓ Target samples with q_type_prob: 10

First 3 target samples:
  Sample 0: [0.001, 0.001, 0.171, 0.76, 0.064, 0.002]
  Sample 1: [0.001, 0.001, 0.171, 0.76, 0.064, 0.002]
  Sample 2: [0.001, 0.001, 0.171, 0.76, 0.064, 0.002]


In [19]:
# 배치 생성 시뮬레이션
print("\n" + "─"*80)
print("🔄 Batch Creation Simulation")
print("─"*80)

# Target 배치 생성 (4개 샘플)
batch_size = 4
target_batch_probs = torch.tensor(target_q_type_probs[:batch_size], dtype=torch.float)

print(f"Target Batch Size: {batch_size}")
print(f"Target Batch q_type_probs shape: {target_batch_probs.shape}")

# Source 배치 샘플링 시뮬레이션
print("\n🔗 Source Sampling (Top-3 Matching):")

source_indices = []
for i, q_type_prob in enumerate(target_batch_probs):
    top3_indices = torch.topk(q_type_prob, k=3).indices.tolist()
    top3_values = torch.topk(q_type_prob, k=3).values.tolist()
    
    print(f"\n  Target {i}:")
    print(f"    Top-3 types: {top3_indices}")
    print(f"    Top-3 probs: {[f'{v:.3f}' for v in top3_values]}")
    print(f"    Matched sources:")
    
    for rank, q_type in enumerate(top3_indices):
        if len(source_q_type_dict[q_type]) > 0:
            source_idx = source_q_type_dict[q_type][rank % len(source_q_type_dict[q_type])]
            source_indices.append(source_idx)
            print(f"      → Type {q_type}: Source[{source_idx}]")
        else:
            print(f"      → Type {q_type}: ⚠️ No source available (skip)")

print(f"\n✓ Total source samples matched: {len(source_indices)}")
print(f"✓ Expected: {batch_size * 3} (최대)")
print(f"✓ Actual: {len(source_indices)}")

# 최종 배치 크기
print("\n" + "─"*80)
print("📦 Final Batch Composition")
print("─"*80)

print(f"Target samples: {batch_size}")
print(f"Source samples: {len(source_indices)}")
print(f"Total batch size: {batch_size + len(source_indices)}")
print(f"Memory ratio: {(batch_size + len(source_indices)) / (batch_size * 2):.2f}x (compared to 1:1 matching)")

# 텐서 형태 확인
print("\n" + "─"*80)
print("📐 Tensor Shapes in Training Loop")
print("─"*80)

# Target 텐서
target_input_ids = torch.tensor([f.input_ids for f in target_features[:batch_size]], dtype=torch.long)
target_q_types = torch.tensor([f.q_type for f in target_features[:batch_size]], dtype=torch.long)

print(f"Target tensors:")
print(f"  • input_ids: {target_input_ids.shape}")
print(f"  • q_types: {target_q_types.shape}")
print(f"  • q_type_probs: {target_batch_probs.shape}")

# Source 텐서 (시뮬레이션)
source_input_ids = torch.tensor([source_features[idx].input_ids for idx in source_indices], dtype=torch.long)
source_q_types = torch.tensor([source_features[idx].q_type for idx in source_indices], dtype=torch.long)

print(f"\nSource tensors:")
print(f"  • input_ids: {source_input_ids.shape}")
print(f"  • q_types: {source_q_types.shape}")

# Concatenated 텐서
concat_input_ids = torch.cat([target_input_ids, source_input_ids], dim=0)
concat_q_types = torch.cat([target_q_types, source_q_types], dim=0)

print(f"\nConcatenated tensors (input to model):")
print(f"  • input_ids: {concat_input_ids.shape}")
print(f"  • q_types: {concat_q_types.shape}")

# Q-type 분포 확인
print("\n" + "─"*80)
print("📊 Q-Type Distribution in Batch")
print("─"*80)

from collections import Counter
target_type_dist = Counter(target_q_types.tolist())
source_type_dist = Counter(source_q_types.tolist())
concat_type_dist = Counter(concat_q_types.tolist())

print("Target batch distribution:")
for q_type in sorted(target_type_dist.keys()):
    print(f"  • Type {q_type}: {target_type_dist[q_type]} samples")

print("\nSource batch distribution:")
for q_type in sorted(source_type_dist.keys()):
    print(f"  • Type {q_type}: {source_type_dist[q_type]} samples")

print("\nCombined batch distribution:")
for q_type in sorted(concat_type_dist.keys()):
    print(f"  • Type {q_type}: {concat_type_dist[q_type]} samples")

print("\n" + "="*80)
print("✅ TRAINING BATCH SIMULATION COMPLETED")
print("="*80)


────────────────────────────────────────────────────────────────────────────────
🔄 Batch Creation Simulation
────────────────────────────────────────────────────────────────────────────────
Target Batch Size: 4
Target Batch q_type_probs shape: torch.Size([4, 6])

🔗 Source Sampling (Top-3 Matching):

  Target 0:
    Top-3 types: [3, 2, 4]
    Top-3 probs: ['0.760', '0.171', '0.064']
    Matched sources:
      → Type 3: Source[0]
      → Type 2: Source[2]
      → Type 4: Source[10]

  Target 1:
    Top-3 types: [3, 2, 4]
    Top-3 probs: ['0.760', '0.171', '0.064']
    Matched sources:
      → Type 3: Source[0]
      → Type 2: Source[2]
      → Type 4: Source[10]

  Target 2:
    Top-3 types: [3, 2, 4]
    Top-3 probs: ['0.760', '0.171', '0.064']
    Matched sources:
      → Type 3: Source[0]
      → Type 2: Source[2]
      → Type 4: Source[10]

  Target 3:
    Top-3 types: [3, 2, 4]
    Top-3 probs: ['0.760', '0.171', '0.064']
    Matched sources:
      → Type 3: Source[0]
      → Type

In [ ]:
def comb_adversarial_training_stage_with_logging(args, target_train_features, target_train_examples, source_train_features,
            eval_features, eval_examples, removed_feature_index, new_generated_train_features, model, epoch,
            device, best_acc, best_f1, logger):
    """
    디버깅 추가 : 실제 학습 루프에서 배치 크기 로깅
    """
    def sample_source_batch_top3(source_data, source_q_type_dict, sample_pointer, q_type_probs):
        """
        q_type_probs 기반 상위 3개 타입에서 소스 샘플링 (로깅 추가)
        """
        output_idx = []
        batch_top3_stats = []  # 각 샘플의 top-3 정보 저장

        for batch_idx, q_type_prob in enumerate(q_type_probs):
            top3_indices = torch.topk(q_type_prob, k=3).indices.tolist()
            top3_values = torch.topk(q_type_prob, k=3).values.tolist()
            
            batch_top3_stats.append({
                "batch_idx": batch_idx,
                "top3_types": top3_indices,
                "top3_probs": top3_values,
                "matched_sources": []
            })

            for q_type in top3_indices:
                if len(source_q_type_dict[q_type]) == 0:
                    batch_top3_stats[-1]["matched_sources"].append(None)
                    continue

                next_q_idx = sample_pointer[q_type] % len(source_q_type_dict[q_type])
                source_idx = source_q_type_dict[q_type][next_q_idx]
                output_idx.append(source_idx)
                batch_top3_stats[-1]["matched_sources"].append(source_idx)
                sample_pointer[q_type] += 1

        input_ids_source, input_masks_source, segment_ids_source, start_positions_source, end_positions_source, \
            q_types_source = [], [], [], [], [], []
        for idx in output_idx:
            input_ids_source.append(source_data[idx][0].unsqueeze(0))
            input_masks_source.append(source_data[idx][1].unsqueeze(0))
            segment_ids_source.append(source_data[idx][2].unsqueeze(0))
            start_positions_source.append(source_data[idx][3].unsqueeze(0))
            end_positions_source.append(source_data[idx][4].unsqueeze(0))
            q_types_source.append(source_data[idx][5].unsqueeze(0))

        return (torch.vstack(input_ids_source), torch.vstack(input_masks_source), torch.vstack(segment_ids_source), \
            torch.cat(start_positions_source, -1), torch.cat(end_positions_source, -1), torch.cat(q_types_source, -1),
            batch_top3_stats)  # 통계 정보도 반환

    # Generate self-training samples
    new_generated_train_features, removed_feature_index = generate_self_training_samples(args, target_train_examples,
        target_train_features, device, model, removed_feature_index, new_generated_train_features, args.generate_prob_th,
        logger)
    if new_generated_train_features is None:
        sys.exit()
    
    logger.info("\n")
    logger.info("====================  Start Adversarial Training Stage  ====================")
    
    # q_type_prob 추출
    all_q_type_probs = []
    fallback_count = 0
    for f in new_generated_train_features:
        if hasattr(f, "q_type_prob") and f.q_type_prob is not None:
            all_q_type_probs.append(f.q_type_prob)
        else:
            prob = [0.0] * 6
            prob[f.q_type] = 1.0
            all_q_type_probs.append(prob)
            fallback_count += 1
    
    # 🔥 로깅: Fallback 사용률
    print(f"\n🔥 RUNTIME LOG - q_type_prob Status:")
    print(f"  Total target features: {len(new_generated_train_features)}")
    print(f"  Fallback (one-hot) count: {fallback_count}")
    print(f"  Proper q_type_prob count: {len(new_generated_train_features) - fallback_count}")
    print(f"  Fallback rate: {fallback_count / len(new_generated_train_features) * 100:.1f}%\n")
    
    all_input_ids = torch.tensor([f.input_ids for f in new_generated_train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in new_generated_train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in new_generated_train_features], dtype=torch.long)
    all_start_positions = torch.tensor([f.start_position for f in new_generated_train_features], dtype=torch.long)
    all_end_positions = torch.tensor([f.end_position for f in new_generated_train_features], dtype=torch.long)
    all_q_types = torch.tensor([f.q_type for f in new_generated_train_features], dtype=torch.long)
    all_q_type_probs = torch.tensor(all_q_type_probs, dtype=torch.float)
    
    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
        all_start_positions, all_end_positions, all_q_types, all_q_type_probs)
    
    # Source data 준비
    source_input_ids = torch.tensor([f.input_ids for f in source_train_features], dtype=torch.long)
    source_input_mask = torch.tensor([f.input_mask for f in source_train_features], dtype=torch.long)
    source_segment_ids = torch.tensor([f.segment_ids for f in source_train_features], dtype=torch.long)
    source_start_positions = torch.tensor([f.start_position for f in source_train_features], dtype=torch.long)
    source_end_positions = torch.tensor([f.end_position for f in source_train_features], dtype=torch.long)
    source_q_types = []
    source_q_type_dict = {0: [], 1: [], 2: [], 3: [], 4: [], 5: []}
    for idx, f in enumerate(source_train_features):
        source_q_types.append(f.q_type)
        source_q_type_dict[f.q_type].append(idx)
    source_q_types = torch.tensor(source_q_types, dtype=torch.long)
    for key in source_q_type_dict.keys():
        random.shuffle(source_q_type_dict[key])
    sample_pointer = [0] * 6
    source_data = TensorDataset(source_input_ids, source_input_mask, source_segment_ids, source_start_positions, 
        source_end_positions, source_q_types)

    # 🔥 로깅: Source 분포
    print(f"🔥 RUNTIME LOG - Source Q-Type Distribution:")
    for q_type in sorted(source_q_type_dict.keys()):
        print(f"  Type {q_type}: {len(source_q_type_dict[q_type])} samples")
    print()

    train_sampler = BERTRandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)
    
    data_len = len(new_generated_train_features)
    logger.info("  Num split examples = %d", data_len)
    logger.info("  Batch size = %d", args.train_batch_size)
    num_train_steps = math.ceil(data_len / args.train_batch_size / args.gradient_accumulation_steps)
    if num_train_steps == 0 and data_len > 0:
        num_train_steps = 1
    t_total = num_train_steps
    logger.info("  Num steps = %d", num_train_steps)

    optimizer_grouped_parameters = get_bert_model_parameters(model)
    optimizer = BertAdam(optimizer_grouped_parameters,
            lr=args.adapt_learning_rate,
            warmup=args.warmup_proportion,
            t_total=t_total)
    global_step = 0

    # 🔥 배치별 로깅 (첫 3개 배치만)
    print(f"\n🔥 RUNTIME LOG - Training Loop (First 3 Batches):")
    print("="*80)

    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        model.train()
        
        # 🔥 로깅: 배치 정보 (첫 3개만)
        if step < 3:
            print(f"\n--- Batch {step} ---")
            print(f"Target batch size: {batch[0].shape[0]}")
            print(f"Sample q_type_prob[0]: {batch[-1][0].tolist()}")
            top3 = torch.topk(batch[-1][0], k=3)
            print(f"Top-3 types: {top3.indices.tolist()}, probs: {[f'{v:.3f}' for v in top3.values.tolist()]}")
        
        # 소스 배치 샘플링
        result = sample_source_batch_top3(source_data, source_q_type_dict, sample_pointer, batch[-1])
        batch_source = result[:6]  # 텐서들만
        batch_stats = result[6]     # 통계 정보
        
        # 🔥 로깅: 소스 배치 크기 (첫 3개만)
        if step < 3:
            print(f"Source batch size: {batch_source[0].shape[0]}")
            print(f"Source sampling ratio: {batch_source[0].shape[0] / batch[0].shape[0]:.2f}x")
            print(f"First target sample matched to sources:")
            first_stats = batch_stats[0]
            for i, (typ, prob, src) in enumerate(zip(first_stats["top3_types"], 
                                                       first_stats["top3_probs"],
                                                       first_stats["matched_sources"])):
                if src is not None:
                    print(f"  Rank {i+1}: Type {typ} (prob={prob:.3f}) → Source[{src}]")
                else:
                    print(f"  Rank {i+1}: Type {typ} (prob={prob:.3f}) → ⚠️  No source")
        
        batch = tuple(t.to(device) for t in batch[:-1])
        batch_source = tuple(t.to(device) for t in batch_source)
        input_ids, input_masks, segment_ids, start_positions, end_positions, q_types = batch
        input_ids_source, input_masks_source, segment_ids_source, start_positions_source, end_positions_source, q_types_source = batch_source
        
        # 타겟 + 소스 concat
        input_ids = torch.cat((input_ids, input_ids_source), 0)
        input_masks = torch.cat((input_masks, input_masks_source), 0)
        segment_ids = torch.cat((segment_ids, segment_ids_source), 0)
        start_positions = torch.cat((start_positions, start_positions_source), 0)
        end_positions = torch.cat((end_positions, end_positions_source), 0)
        q_types = torch.cat((q_types, q_types_source), 0)
        
        # 🔥 로깅: 최종 배치 크기 (첫 3개만)
        if step < 3:
            print(f"Combined batch size: {input_ids.shape[0]}")
            from collections import Counter
            q_type_dist = Counter(q_types.cpu().tolist())
            print(f"Q-type distribution: {dict(sorted(q_type_dist.items()))}")
        
        # QC4QA loss 계산
        loss = model.forward_ours(input_ids, segment_ids, input_masks, start_positions,
                end_positions, q_types, lambda_c=args.lambda_c)
        if args.gradient_accumulation_steps > 1:
            loss = loss / args.gradient_accumulation_steps
        
        loss.backward()
        
        if (step + 1) % args.gradient_accumulation_steps == 0:
            lr_this_step = args.adapt_learning_rate * warmup_linear(global_step / t_total, args.warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group["lr"] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
    
    print("\n" + "="*80)
    print("🔥 TRAINING LOOP COMPLETED")
    print("="*80 + "\n")
        
    final_acc, final_f1 = None, None
    if epoch == args.num_train_epochs - 1:
        final_acc, final_f1 = evaluation_stage(args, eval_examples, eval_features, device, model,
            global_step=global_step, best_acc=None, best_f1=None, logger=logger)
        best_acc, best_f1 = compare_performance(args, best_acc, best_f1, final_acc, final_f1, model, logger)
    else:
        best_acc, best_f1 = evaluation_stage(args, eval_examples, eval_features, device, model,
            global_step=global_step, best_acc=best_acc, best_f1=best_f1, logger=logger)
    return best_acc, best_f1, final_acc, final_f1

🔥 ADDING RUNTIME LOGGING TO TRAINING LOOP
✅ Runtime logging function defined.

To use it, run:
  globals()['comb_adversarial_training_stage'] = comb_adversarial_training_stage_with_logging
  adaptation_stage(args, tokenizer, device, logger, debug=True)


In [32]:
# 로깅 함수로 덮어쓰기
globals()['comb_adversarial_training_stage'] = comb_adversarial_training_stage_with_logging
model = prepare_model(args, device)
adaptation_stage(args, tokenizer, device, logger, debug=True)

/tmp/ipykernel_2255560/1948527526.py:360: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(input_model_file)
11/04/2025 15:10:04 - INFO - pytorch_


🔥 RUNTIME LOG - q_type_prob Status:
  Total target features: 2
  Fallback (one-hot) count: 0
  Proper q_type_prob count: 2
  Fallback rate: 0.0%

🔥 RUNTIME LOG - Source Q-Type Distribution:
  Type 0: 0 samples
  Type 1: 0 samples
  Type 2: 11 samples
  Type 3: 18 samples
  Type 4: 3 samples
  Type 5: 18 samples


🔥 RUNTIME LOG - Training Loop (First 3 Batches):


Iteration: 100%|██████████| 1/1 [00:00<00:00,  6.67it/s]
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -   ***** Running Predictions *****
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -     Num orig examples = 50
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -     Num split examples = 50
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -     Batch size = 12
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -   Start evaluating



--- Batch 0 ---
Target batch size: 2
Sample q_type_prob[0]: [0.0010000000474974513, 0.010999999940395355, 0.3059999942779541, 0.6769999861717224, 0.003000000026077032, 0.0010000000474974513]
Top-3 types: [3, 2, 1], probs: ['0.677', '0.306', '0.011']
Source batch size: 5
Source sampling ratio: 2.50x
First target sample matched to sources:
  Rank 1: Type 3 (prob=0.677) → Source[46]
  Rank 2: Type 2 (prob=0.306) → Source[14]
  Rank 3: Type 1 (prob=0.011) → ⚠️  No source
Combined batch size: 7
Q-type distribution: {2: 3, 3: 3, 5: 1}

🔥 TRAINING LOOP COMPLETED



Evaluating: 100%|██████████| 5/5 [00:00<00:00, 20.15it/s]
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -   Writing predictions to: model/squad2target/predictions_1.json
11/04/2025 15:12:01 - INFO - utils.ConfigLogger -   Writing nbest to: model/squad2target/nbest_predictions_1.json
/tmp/ipykernel_2255560/1948527526.py:27: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(message)
11/04/2025 15:12:01 - WARNING - utils.ConfigLogger -   Unanswered question validation/fd0dc9fcbb41450c388f6d3f90153da19f53e050 will receive score 0.
11/04/2025 15:12:01 - WARNING - utils.ConfigLogger -   Unanswered question validation/7ad6bf269adef016560ea4d857150771842e0d23 will receive score 0.
11/04/2025 15:12:01 - WARNING - utils.ConfigLogger -   Unanswered question validation/de308d2ec5722143d8156c3fe2192a2884ca86f9 will receive score 0.
11/04/2025 15:12:01 - WARNING - utils.ConfigLogger -   Unanswered question validation/c1875aa38c8ac079d128a4ddf92259479955abcc w


🔥 RUNTIME LOG - q_type_prob Status:
  Total target features: 2
  Fallback (one-hot) count: 0
  Proper q_type_prob count: 2
  Fallback rate: 0.0%

🔥 RUNTIME LOG - Source Q-Type Distribution:
  Type 0: 0 samples
  Type 1: 0 samples
  Type 2: 11 samples
  Type 3: 18 samples
  Type 4: 3 samples
  Type 5: 18 samples


🔥 RUNTIME LOG - Training Loop (First 3 Batches):


Iteration: 100%|██████████| 1/1 [00:00<00:00,  6.35it/s]
11/04/2025 15:12:02 - INFO - utils.ConfigLogger -   ***** Running Predictions *****
11/04/2025 15:12:02 - INFO - utils.ConfigLogger -     Num orig examples = 50
11/04/2025 15:12:02 - INFO - utils.ConfigLogger -     Num split examples = 50
11/04/2025 15:12:02 - INFO - utils.ConfigLogger -     Batch size = 12
11/04/2025 15:12:02 - INFO - utils.ConfigLogger -   Start evaluating



--- Batch 0 ---
Target batch size: 2
Sample q_type_prob[0]: [0.0010000000474974513, 0.010999999940395355, 0.3059999942779541, 0.6769999861717224, 0.003000000026077032, 0.0010000000474974513]
Top-3 types: [3, 2, 1], probs: ['0.677', '0.306', '0.011']
Source batch size: 5
Source sampling ratio: 2.50x
First target sample matched to sources:
  Rank 1: Type 3 (prob=0.677) → Source[35]
  Rank 2: Type 2 (prob=0.306) → Source[2]
  Rank 3: Type 1 (prob=0.011) → ⚠️  No source
Combined batch size: 7
Q-type distribution: {2: 3, 3: 3, 5: 1}

🔥 TRAINING LOOP COMPLETED



Evaluating: 100%|██████████| 5/5 [00:00<00:00, 20.11it/s]
11/04/2025 15:12:03 - INFO - utils.ConfigLogger -   Writing predictions to: model/squad2target/predictions_1.json
11/04/2025 15:12:03 - INFO - utils.ConfigLogger -   Writing nbest to: model/squad2target/nbest_predictions_1.json
11/04/2025 15:12:03 - WARNING - utils.ConfigLogger -   Unanswered question validation/fd0dc9fcbb41450c388f6d3f90153da19f53e050 will receive score 0.
11/04/2025 15:12:03 - WARNING - utils.ConfigLogger -   Unanswered question validation/7ad6bf269adef016560ea4d857150771842e0d23 will receive score 0.
11/04/2025 15:12:03 - WARNING - utils.ConfigLogger -   Unanswered question validation/de308d2ec5722143d8156c3fe2192a2884ca86f9 will receive score 0.
11/04/2025 15:12:03 - WARNING - utils.ConfigLogger -   Unanswered question validation/c1875aa38c8ac079d128a4ddf92259479955abcc will receive score 0.
11/04/2025 15:12:03 - WARNING - utils.ConfigLogger -   Unanswered question validation/1993fab60212e6627aea8ae78a33d9a8